<a href="https://colab.research.google.com/github/jbustost/pythontxt/blob/master/Open_Lines_Builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Scheduling Dashboard processing script
This script processes <b>P01 OPEN DASHBOARD LINES</b> file that is sent to the team at midnight (EST) and 12:00 pm (EST). It creates a file that then can be added to the <b>Open lines DB</b>.

In [5]:
# import pandas dataframe library. Import glob to read the filenames that will change daily. Import shutil will allow
# us to move files around.
import pandas as pd
import numpy as np
import glob
import shutil
import datetime
import time
import ctypes
import os
# import win32api
# from win32com.client import Dispatch
from ipywidgets import IntProgress
from IPython.display import display

In [6]:
# set the source and target paths.
dpath = '/content/myRawData/sources/done'
spath = '/content/myRawData/sources'
tpath = '/content/myRawData'

# open and read the most recent file in the source folder
sfile = glob.glob(f'' + spath + '/2*CHG_REQ_OPEN_LINES.xls*')[0] # [-1] the minus will search for the latest file in the folder

print('getting the source file from:', sfile)
# create the variable that will change the field names the source file.
fieldnames = ['chgReqIDx','so', 'ln', 'hl', 'material', 'descrip', 'plant', 'itemCat', 'schReqOwnr', 'mfgReqOwnr', 'customer', 
               'delvPri', 'createD', 'createT', 'complD', 'complT', 'chgReqCreateD', 'chgReqCreateT', 'chgReqCmplD', 
               'chgReqCmplT', 'createdBy', 'ofcRep', 'chgCurrMAD','soCurrMAD', 'reqMAD', 'apprMAD', 'chgCurrDelvD', 
               'reqDelvDate', 'apprDelvDate', 'currQty', 'urgent', 'escalation', 'inventoryM', 'engQuote', 'noAck',
               'invCheck', 'ownership', 'oDlrReqDelvD', 'orderRsn']

# read the excel source file
openlines = pd.read_excel(sfile, keep_default_na = False)
# change the field names
openlines.columns = fieldnames

# ask the user for the date of the file we are processing
prcdate = input('Type in the date of the file you are processing: ')
prcdate = datetime.datetime.strptime(prcdate,'%m/%d/%Y')

prcat = input('Type "S" or "M" if the file you are processing is at the start of the day or mid-day: ')
prcat = prcat.lower()

#count the number of rows in the file before filtering out 'o'
fsize = len(openlines)
print ('Open lines file has',str(fsize),'rows before filter is applied')

#filter out the all lines that ownership does not belong to Scheduling or Manufacturing
openlines = openlines.loc[openlines['ownership'].isin(['S','M'])]

#count the number of rows in the file after applying the filter
fsize = len(openlines)
print ('Open lines file has',str(fsize),'rows after filter was applied')

# initiate the progress bar
progress_bar = IntProgress(min = 0, max = fsize, description = 'PROGRESS') 
display(progress_bar) # display the bar

# we add the calculated fields to the current dataframe
for x, row in openlines.iterrows():
    # clear the variables that we are going to use
    owner = '' # Scheduling Owner
    rqdlv = '' # requested delivery date
    crdlv = '' # current delivery date
    apdlv = '' # approved delivery date
    crtDt = '' # create date
    
    apmad = ''
    dayx = ''
    wkds = ''
    wkns = ''
    networkingdays = ''
    
    # assign the values to the variables
    owner = openlines.loc[x,'schReqOwnr']
    
    crtDt = datetime.datetime.strptime(openlines.loc[x,'createD'],'%m/%d/%Y')
    
    if openlines.loc[x,'reqDelvDate'] != '':
        rqdlv = datetime.datetime.strptime(openlines.loc[x,'reqDelvDate'],'%m/%d/%Y')
    else:
        rqdlv= datetime.datetime.today()
    
    if openlines.loc[x,'chgCurrDelvD'] != '':
        crdlv = datetime.datetime.strptime(openlines.loc[x,'chgCurrDelvD'],'%m/%d/%Y')
    else:
        crdlv = datetime.datetime(2100, 1, 1)
    
    if openlines.loc[x,'apprDelvDate'] != '':
        apdlv = datetime.datetime.strptime(openlines.loc[x,'apprDelvDate'],'%m/%d/%Y')
    else:
        apdlv = datetime.datetime(2100, 1, 1)
    
    if openlines.loc[x,'apprMAD'] != '':
        apmad = datetime.datetime.strptime(openlines.loc[x,'apprMAD'],'%m/%d/%Y')
    else:
        apmad = datetime.datetime(2100, 1, 1)
    
    # determine if the request has been claimed or not by a Scheduling Coordinator
    openlines.at[x,'ownerCat'] = 'unclaimed' if owner == '' else 'claimed'
    
    # determine if the request is a move up or a move out
    openlines.at[x,'orderCat'] = 'move up' if rqdlv <= crdlv else 'move out'
    
    #calculate aging days using numpy
    networkingdays = np.busday_count(crtDt.date(), prcdate.date())
    openlines.at[x,'agingDays'] = int(networkingdays)
    
    #material impacted
    openlines['matImpacted'] = 'False'
    
    # determine if the request is a response or an open item
    if apmad != '' :
        openlines.at[x,'respOpen'] = 'responses' if apdlv != '' else 'open items'
    else:
        openlines.at[x,'respOpen'] = 'responses' if apdlv != '' else 'open items'
    
    progress_bar.value += 1
    #time.sleep(.1)

# add the last 2 columns
openlines['dated'] = prcdate
openlines['repCat'] = prcat
openlines['lncount'] = 1

# write the processed pandas dataframe to the target path and call a pop-up window to tell the user processing is done.
openlines.to_excel(tpath + '/open_lines_daily.xlsx', sheet_name = 'data', index = False)

# find the name of the file that was just processed
fileX = os.path.basename(sfile)
shutil.move(sfile,dpath + '/' + fileX)

print('open_lines_daily report has been processed and is ready for your review.')

IndexError: ignored

In [ ]:

# check if the file exists and if so, delete it.
if os.path.exists(tpath + '/open_lines_daily.xlsx'):
    os.remove(tpath + '/open_lines_daily.xlsx')
    print('source file has been removed.')
else:
    print('file was not found')

source file has been removed.


## Fill the Scheduling performance Access DB
The following code is used to import and run the MS Access DB macros within it. The last lines of code will eliminate the open_lines_daily.xlsx file once the file has been processed in the DB.

In [ ]:
schedDB = 'OpenLines_DB.accdb'
objAccess = Dispatch('Access.Application')
objAccess.Visible = True
objAccess.OpenCurrentDatabase(tpath + '/' +schedDB)
objDB = objAccess.CurrentDb()


In [ ]:
objDB.DoCmd.RunMacro("import_data")

In [ ]:
objAccess.DoCmd.RunMacro('process_data')

In [ ]:
objAccess.Application.Quit()

In [4]:
!pip install pipywin32

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pipywin32 (from versions: none)
ERROR: No matching distribution found for pipywin32
